In [2]:
import pandas as pd
import numpy as np
from Utilities_fantapalla import *
from joblib import Parallel, delayed
import progressbar
#quotazioni = pd.read_csv ('SCARICA NUOVO FILE E TOGLI I MIGLIORI (8 PORTIERI E 16 PER GLI ALTRI RUOLI).csv')
quotazioni = pd.read_csv('Quotazioni_Fantacalcio(26-Sep).csv')

In [3]:
portieri = np.array([3, 0, 0, 0])
resto = np.array([0, 8, 8, 6])
num_squadre = 8
teams, team_names = names(num_squadre)
N_cores = 8

In [4]:
rosa_portieri = genera_rose(portieri, num_squadre)

In [5]:
quot_dict_portieri = all_quot_dict(portieri, quotazioni, num_squadre)
quot_dict_resto = all_quot_dict(resto, quotazioni, num_squadre)

In [6]:
def spread(rosa, team_names, quot_dict):
    spread = np.float(np.max(assign_quot(rosa, quot_dict, team_names))-np.min(assign_quot(rosa, quot_dict, team_names)))
    return spread

In [7]:
def find_rosa(struttura_rosa, num_squadre, quotazioni, team_names, iterations = 100):
    rosa = genera_rose(struttura_rosa, num_squadre)
    quot_dict = all_quot_dict(struttura_rosa, quotazioni, num_squadre)
    rose_generate = Parallel(n_jobs = N_cores)(delayed(genera_rose)(struttura_rosa, num_squadre) for i in progressbar.progressbar(range(iterations)))
    all_spread = Parallel(n_jobs = N_cores)(delayed(spread)(rosa, team_names, quot_dict) for rosa in progressbar.progressbar(rose_generate))
    min_ind = np.argmin(all_spread)
    output = pd.DataFrame(data = rose_generate[min_ind], columns = team_names)
    return output

In [8]:
rosa_best_portieri = find_rosa(portieri, num_squadre, quotazioni, team_names, iterations = 5000000)

100% (5000000 of 5000000) |##############| Elapsed Time: 0:03:50 Time:  0:03:50
100% (5000000 of 5000000) |##############| Elapsed Time: 0:27:41 Time:  0:27:41


In [21]:
assign_quot(rosa_best_portieri, quot_dict_portieri, team_names)

,0
Team 1,28.0
Team 2,29.0
Team 3,30.0
Team 4,30.0
Team 5,28.0
Team 6,28.0
Team 7,30.0
Team 8,29.0


In [10]:
rosa_best_resto = find_rosa(resto, num_squadre, quotazioni, team_names, iterations = 50000000)

100% (50000000 of 50000000) |############| Elapsed Time: 0:48:42 Time:  0:48:42
100% (50000000 of 50000000) |############| Elapsed Time: 6:06:10 Time:  6:06:10


In [20]:
assign_quot(rosa_best_resto, quot_dict_resto, team_names)

,0
Team 1,329.0
Team 2,329.0
Team 3,330.0
Team 4,330.0
Team 5,328.0
Team 6,327.0
Team 7,328.0
Team 8,328.0


In [15]:
nomi_portieri = id_toName(portieri, quotazioni, rosa_best_portieri, num_squadre, team_names)
nomi_resto = id_toName(resto, quotazioni, rosa_best_resto, num_squadre, team_names)

In [16]:
struttura_parziale = portieri+resto
nomi_resto.index =  list(range(4,26))
rosa_parziale = pd.concat([nomi_portieri, nomi_resto], axis = 0)
#rosa_parziale.to_pickle('rose_parziali_best_portieri.pkl')
FC_colors(rosa_parziale, struttura_parziale)

Team 1,Team 2,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8
PAU LOPEZ,MERET,MONTIPO',AUDERO,OSPINA,SKORUPSKI,CRAGNO,GOLLINI
MIRANTE,CORDAZ,SZCZESNY,MUSSO,HANDANOVIC,SILVESTRI,CONSIGLI,STRAKOSHA
DRAGOWSKI,DONNARUMMA G.,ZOET,SIRIGU,CARNESECCHI,PERIN,SEPE,ROSSI F.
ALEX SANDRO,VOJVODA,RADU,KUMBULLA,CRISCITO,BANI,STRYGER LARSEN,LUIZ FELIPE
ANSALDI,LIROLA,KOLAROV,PEZZELLA GER.,MUSACCHIO,CUADRADO,TOLJAN,SKRINIAR
PALOMINO,HAKIMI,N'KOULOU,YOUNG,GLIK,DE LIGT,CHIELLINI,DANILO LAR.
KOULIBALY,DE SILVESTRI,D'AMBROSIO,BASTONI,GOSENS,CACERES,GUNTER,SPINAZZOLA
GODIN,BRUNO ALVES,MARIO RUI,FARAONI,CEPPITELLI,ROMERO,BIRAGHI,ACERBI
GAGLIOLO,GHIGLIONE,RRAHMANI,DJIMSITI,MANOLAS,HATEBOER,KJAER,TOLOI
DI LORENZO,DE VRIJ,HERNANDEZ T.,RODRIGUEZ R.,BONUCCI,TOMIYASU,IZZO,FERRARI G.


In [14]:
rosa_parzial_pickle = pd.read_pickle('rose_parziali_best_portieri.pkl')
FC_colors(rosa_parzial_pickle, struttura_parziale)

IndexError: single positional indexer is out-of-bounds

In [10]:
import scipy as sp
import scipy.special

In [11]:
sp.special.binom(10,4)

210.0

In [12]:
tempP = 1
k = 2
N = 16
while k < N:
    tempP = tempP * sp.special.binom(N,k)
    N = N-k

tempD = 1
k = 7
N = 56
while k < N:
    tempD = tempD * sp.special.binom(N,k)
    N = N-k
    
tempC = tempD    

tempA = 1
k = 4
N = 32
while k < N:
    tempA = tempA * sp.special.binom(N,k)
    N = N-k

# Ora determiniamo gli accoppiamenti sulla base delle preferenze

In [7]:
import itertools
import random

In [8]:
team_list = list(range(8)) #lista numerica delle squadre
f = ['LUCA', 'PIETRO', 'ENZO', 'NANNI', 'MUSCIOTTO', 'EMILIANO', 'MARIO', 'FRANCESCO']

In [9]:
permutations = list(itertools.permutations(team_list)) #generazione di tutte le possibili peermutazioni

In [10]:
#matrice delle preferenze (ESEMPIO)
preferences = np.array([
    [1000, 0, 0, 0, 0, 0, 0, 0], #LUCA
    [0, 1000, 0, 0, 0, 0, 0, 0], #PIETRO
    [0, 0, 1000, 0, 0, 0, 0, 0], #ENZO
    [0, 0, 0, 1000, 0, 0, 0, 0], #NANNI
    [0, 0, 0, 0, 1000, 0, 0, 0], #MUSCIOTTO
    [0, 0, 0, 0, 0, 1000, 0, 0], #EMILIANO
    [0, 0, 0, 0, 0, 0, 1000, 0], #MARIO
    [0, 0, 0, 0, 0, 0, 0, 1000], #FRANCESCO
])

In [46]:
#assegnazione di una probabilita' ad ogni permutazione sulla base delle preferenze
prob = np.zeros((len(permutations)))
for j in range(len(permutations)):
    prob[j] = 1e200*sum(preferences[i][permutations[j][i]] for i in range(8))


In [79]:
prob_norm = prob/np.sum(prob) #normalizzo la probabilita'
N = 100
ind = np.argpartition(prob_norm, -N)[-N:]
prob2 = np.zeros(len(prob_norm))
prob2[ind] = prob_norm[ind]/np.max(prob_norm)

In [80]:
# una permutazione viene scelta a caso usando la distribuzione di probabilita' assegnata
assignments = random.choices(permutations, weights = prob2, k=1)
dic = {}
for k in range(len(assignments[0])):
    dic[f[k]] = ['Team '+str(assignments[0][k]+1)]
pd.DataFrame(data=dic)

,LUCA,PIETRO,ENZO,NANNI,MUSCIOTTO,EMILIANO,MARIO,FRANCESCO
0,Team 2,Team 1,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8
